In [ ]:
# Customer Support Voice Chatbot - Local Testing Notebook
# This notebook allows you to test the core LLM functionality before building the complete application

import os
import numpy as np
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import whisper
from langchain.llms import HuggingFacePipeline
from langchain.chains import ConversationChain, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader, CSVLoader, DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma, FAISS
import requests
from IPython.display import Audio, display
import matplotlib.pyplot as plt
from scipy.io import wavfile
import io
from tqdm.notebook import tqdm

# Set this to True if you have a GPU, False otherwise
has_gpu = torch.cuda.is_available()
print(f"GPU Available: {has_gpu}")

# Set maximum GPU memory usage (adjust based on your GPU)
if has_gpu:
    torch.cuda.set_per_process_memory_fraction(0.9)

# PART 1: SETUP MODELS

def load_llm(model_id="HuggingFaceH4/zephyr-7b-beta", use_8bit=True):
    """Load a language model for text generation."""
    print(f"Loading model: {model_id}")
    
    # Device mapping strategy
    if has_gpu:
        device_map = "auto"
    else:
        device_map = "cpu"
        use_8bit = False  # 8-bit quantization requires GPU
    
    # Determine the data type
    dtype = torch.float16 if has_gpu else torch.float32
    
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    
    # Load model with appropriate quantization
    if use_8bit:
        print("Loading with 8-bit quantization")
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map=device_map,
            load_in_8bit=True,
            torch_dtype=dtype
        )
    else:
        print(f"Loading with {dtype} precision")
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map=device_map,
            torch_dtype=dtype
        )
    
    # Create a text generation pipeline
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_length=512,
        temperature=0.7,
        top_p=0.95,
        repetition_penalty=1.2
    )
    
    # Create a LangChain wrapper for the pipeline
    llm = HuggingFacePipeline(pipeline=pipe)
    return llm

def load_whisper_model(model_size="base"):
    """Load the Whisper model for speech recognition."""
    print(f"Loading Whisper model: {model_size}")
    model = whisper.load_model(model_size)
    return model

# PART 2: PREPARE CUSTOMER SUPPORT DATA

def download_customer_support_dataset():
    """Download a customer support dataset if not already available."""
    # Check for Bitext dataset on Hugging Face
    try:
        from datasets import load_dataset
        dataset = load_dataset("bitext/Bitext-customer-support-llm-chatbot-training-dataset")
        df = pd.DataFrame(dataset['train'])
        print(f"Loaded dataset with {len(df)} examples")
        return df
    except Exception as e:
        print(f"Error loading dataset from Hugging Face: {e}")
        
        # Fallback: Try to download the Customer Support on Twitter dataset
        print("Trying to download Customer Support on Twitter dataset...")
        url = "https://github.com/thoughtvector/customer-support-on-twitter/raw/master/twcs.csv"
        try:
            df = pd.read_csv(url)
            print(f"Loaded Twitter dataset with {len(df)} examples")
            return df
        except Exception as e2:
            print(f"Error downloading Twitter dataset: {e2}")
            
            # Create a minimal synthetic dataset as a last resort
            print("Creating minimal synthetic dataset for testing")
            data = {
                'instruction': [
                    "How do I reset my password?",
                    "Where is my order?",
                    "I want to cancel my subscription",
                    "Can I get a refund?",
                    "The product is defective"
                ],
                'response': [
                    "To reset your password, please go to the login page and click on 'Forgot Password'. You'll receive an email with instructions.",
                    "You can track your order by logging into your account and going to the 'Orders' section. If you need more help, please provide your order number.",
                    "To cancel your subscription, go to 'Account Settings' and select 'Subscription'. Click on 'Cancel Subscription' and follow the prompts.",
                    "Refunds can be requested within 30 days of purchase. Please go to 'Orders', find your order, and click 'Request Refund'.",
                    "I'm sorry to hear that. Please contact our support team with your order number and a description of the issue."
                ]
            }
            return pd.DataFrame(data)

def prepare_vector_store(df, column_names=None):
    """Create a vector store from the dataset for retrieval."""
    if column_names is None:
        # Try to guess the relevant columns
        if 'instruction' in df.columns and 'response' in df.columns:
            column_names = ['instruction', 'response']
        elif 'query' in df.columns and 'response' in df.columns:
            column_names = ['query', 'response']
        elif 'text' in df.columns:
            column_names = ['text']
        elif 'content' in df.columns:
            column_names = ['content']
        else:
            print(f"Available columns: {df.columns.tolist()}")
            raise ValueError("Could not determine which columns to use. Please specify column_names.")
    
    print(f"Using columns: {column_names}")
    
    # Combine the specified columns into documents
    documents = []
    for i, row in df.iterrows():
        content = ""
        for col in column_names:
            if col in row and pd.notna(row[col]):
                content += f"{col.upper()}: {row[col]}\n"
        
        if content:
            documents.append({"content": content, "source": f"row_{i}"})
    
    print(f"Created {len(documents)} documents")
    
    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    
    chunks = []
    for doc in documents:
        doc_chunks = text_splitter.split_text(doc['content'])
        for chunk in doc_chunks:
            chunks.append({
                'content': chunk,
                'source': doc['source']
            })
    
    print(f"Split into {len(chunks)} chunks")
    
    # Initialize embeddings
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    
    # Create vector store
    texts = [chunk['content'] for chunk in chunks]
    metadatas = [{'source': chunk['source']} for chunk in chunks]
    
    vector_store = FAISS.from_texts(
        texts=texts,
        embedding=embeddings,
        metadatas=metadatas
    )
    
    return vector_store

# PART 3: CREATE CONVERSATION CHAIN

def create_simple_chain(llm):
    """Create a simple conversation chain without retrieval."""
    # Create a conversation memory
    memory = ConversationBufferMemory(return_messages=True)
    
    # Define the prompt template for customer support
    template = """
    You are a helpful customer support assistant.
    Be friendly, concise, and helpful. If you don't know the answer,
    simply say you'll escalate to a human agent.
    
    Current conversation:
    {history}
    
    Customer: {input}
    AI Assistant:"""
    
    prompt = PromptTemplate(
        input_variables=["history", "input"],
        template=template
    )
    
    # Create the conversation chain
    conversation = ConversationChain(
        llm=llm,
        memory=memory,
        prompt=prompt,
        verbose=True
    )
    
    return conversation

def create_retrieval_chain(llm, vector_store):
    """Create a conversation chain with retrieval augmentation."""
    # Create a conversation memory
    memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True
    )
    
    # Define the prompt template
    template = """
    You are a helpful customer support assistant.
    Be friendly, concise, and helpful. If you don't know the answer,
    simply say you'll escalate to a human agent.
    
    Here is some context that might be relevant to the customer's question:
    {context}
    
    Current conversation:
    {chat_history}
    
    Customer: {question}
    AI Assistant:"""
    
    prompt = PromptTemplate(
        input_variables=["context", "chat_history", "question"],
        template=template
    )
    
    # Create the conversation chain with retrieval
    retrieval_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vector_store.as_retriever(),
        memory=memory,
        combine_docs_chain_kwargs={"prompt": prompt},
        verbose=True
    )
    
    return retrieval_chain

# PART 4: SPEECH-TO-TEXT & TEXT-TO-SPEECH TESTING

def transcribe_audio_file(whisper_model, audio_path):
    """Transcribe an audio file using Whisper."""
    result = whisper_model.transcribe(audio_path)
    return result["text"]

def transcribe_audio_bytes(whisper_model, audio_bytes):
    """Transcribe audio bytes using Whisper."""
    # Convert bytes to a temporary file
    with open("temp_audio.wav", "wb") as f:
        f.write(audio_bytes)
    
    # Transcribe
    result = whisper_model.transcribe("temp_audio.wav")
    
    # Clean up
    os.remove("temp_audio.wav")
    
    return result["text"]

def test_tts_with_edge_tts():
    """Test text-to-speech using Edge-TTS (Microsoft Edge's TTS service)."""
    try:
        # This requires edge-tts: pip install edge-tts
        import edge_tts
        import asyncio
        
        async def synthesize_speech(text, output_file="temp_output.wav"):
            voice = "en-US-SaraNeural"
            communicate = edge_tts.Communicate(text, voice)
            await communicate.save(output_file)
            return output_file
        
        # Test the TTS
        test_text = "Hello, this is a test of the text-to-speech system. How can I help you today?"
        output_file = asyncio.run(synthesize_speech(test_text))
        
        # Play the audio
        display(Audio(output_file))
        
        print("TTS test successful!")
        return True
    except Exception as e:
        print(f"Error testing TTS: {e}")
        print("Please install edge-tts: pip install edge-tts")
        return False

# PART 5: TEST THE COMPLETE PIPELINE

def run_test_conversation(chain, questions=None):
    """Run a test conversation with predefined questions."""
    if questions is None:
        questions = [
            "Hi, I need help with my account",
            "I can't log in to my account",
            "I forgot my password",
            "How do I reset it?",
            "Thanks for your help!"
        ]
    
    print("Starting test conversation...")
    responses = []
    
    for question in questions:
        print(f"\nCustomer: {question}")
        
        # Check the type of chain to determine how to call it
        if isinstance(chain, ConversationalRetrievalChain):
            response = chain({"question": question})
            answer = response["answer"]
        else:
            answer = chain.predict(input=question)
        
        print(f"AI: {answer}")
        responses.append(answer)
    
    return responses

def evaluate_responses(responses, criteria=None):
    """Evaluate the quality of responses based on predefined criteria."""
    if criteria is None:
        criteria = {
            "Helpfulness": "Does the response address the customer's question?",
            "Clarity": "Is the response clear and easy to understand?",
            "Tone": "Is the tone friendly and supportive?",
            "Conciseness": "Is the response appropriately concise?"
        }
    
    print("\nResponse Evaluation:")
    
    evaluations = {}
    for criterion, description in criteria.items():
        print(f"\n{criterion} ({description}):")
        scores = []
        for i, response in enumerate(responses):
            # You can implement automatic scoring here, or just ask for manual input
            try:
                score = float(input(f"Rate response {i+1} (0-10): "))
                scores.append(score)
            except ValueError:
                print("Please enter a number between 0 and 10")
                score = float(input(f"Rate response {i+1} (0-10): "))
                scores.append(score)
        
        avg_score = sum(scores) / len(scores)
        evaluations[criterion] = avg_score
        print(f"Average {criterion} score: {avg_score:.2f}/10")
    
    # Overall score
    overall = sum(evaluations.values()) / len(evaluations)
    print(f"\nOverall evaluation score: {overall:.2f}/10")
    
    return evaluations

# PART 6: MAIN TESTING FUNCTION

def main():
    """Main function to run the notebook."""
    # Step 1: Load the LLM
    model_options = {
        "1": "HuggingFaceH4/zephyr-7b-beta",
        "2": "mistralai/Mistral-7B-Instruct-v0.1",
        "3": "google/gemma-2b-it",
        "4": "TheBloke/Llama-2-7B-Chat-GPTQ",
    }
    
    print("Select a model to load:")
    for key, value in model_options.items():
        print(f"{key}: {value}")
    
    model_choice = input("Enter your choice (1-4): ")
    model_id = model_options.get(model_choice, model_options["1"])
    
    llm = load_llm(model_id)
    
    # Step 2: Load and prepare data
    print("\nLoading customer support dataset...")
    df = download_customer_support_dataset()
    
    # Display sample data
    print("\nSample data:")
    print(df.head())
    
    # Step 3: Create vector store
    print("\nCreating vector store for retrieval...")
    vector_store = prepare_vector_store(df)
    
    # Step 4: Create conversation chains
    print("\nCreating conversation chains...")
    simple_chain = create_simple_chain(llm)
    retrieval_chain = create_retrieval_chain(llm, vector_store)
    
    # Step 5: Test the chains
    print("\nWhich chain would you like to test?")
    print("1: Simple conversation chain")
    print("2: Retrieval-augmented chain")
    
    chain_choice = input("Enter your choice (1-2): ")
    
    chain = retrieval_chain if chain_choice == "2" else simple_chain
    
    # Custom test questions
    print("\nWould you like to use custom test questions? (y/n)")
    custom_questions = input().lower() == 'y'
    
    if custom_questions:
        print("Enter your questions (one per line, enter an empty line to finish):")
        questions = []
        while True:
            question = input()
            if not question:
                break
            questions.append(question)
    else:
        questions = None
    
    # Run the test
    responses = run_test_conversation(chain, questions)
    
    # Step 6: Evaluate the responses
    print("\nWould you like to evaluate the responses? (y/n)")
    evaluate = input().lower() == 'y'
    
    if evaluate:
        evaluations = evaluate_responses(responses)
    
    # Step 7: Test speech components (optional)
    print("\nWould you like to test speech-to-text and text-to-speech? (y/n)")
    test_speech = input().lower() == 'y'
    
    if test_speech:
        # Load Whisper model
        whisper_model = load_whisper_model()
        
        # Test TTS
        test_tts_with_edge_tts()
    
    print("\nTesting complete!")

if __name__ == "__main__":
    main()

GPU Available: False
Select a model to load:
1: HuggingFaceH4/zephyr-7b-beta
2: mistralai/Mistral-7B-Instruct-v0.1
3: google/gemma-2b-it
4: TheBloke/Llama-2-7B-Chat-GPTQ


In [2]:
# Automated Customer Support Chatbot Testing with Open-Access Models
# This notebook uses fully open models that don't require Hugging Face login

import os
import numpy as np
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from IPython.display import display
from tqdm.notebook import tqdm

print("Starting automated customer support chatbot testing...")

# Check for GPU availability
has_gpu = torch.cuda.is_available()
print(f"GPU Available: {has_gpu}")

# PART 1: CREATE SYNTHETIC DATASET (LIGHTWEIGHT)

print("\nCreating minimal synthetic dataset...")
data = {
    'instruction': [
        "How do I reset my password?",
        "Where is my order?",
        "I want to cancel my subscription",
        "Can I get a refund?",
        "The product is defective",
        "How do I contact support?",
        "I need to update my payment method",
        "I can't log into my account",
        "Is there a free trial available?",
        "How do I track my order?"
    ],
    'response': [
        "To reset your password, please go to the login page and click on 'Forgot Password'. You'll receive an email with instructions.",
        "You can track your order by logging into your account and going to the 'Orders' section. If you need more help, please provide your order number.",
        "To cancel your subscription, go to 'Account Settings' and select 'Subscription'. Click on 'Cancel Subscription' and follow the prompts.",
        "Refunds can be requested within 30 days of purchase. Please go to 'Orders', find your order, and click 'Request Refund'.",
        "I'm sorry to hear that. Please contact our support team with your order number and a description of the issue.",
        "You can contact our support team by email at support@example.com or by phone at 1-800-123-4567. Our hours are Monday to Friday, 9 AM to 5 PM.",
        "To update your payment method, go to 'Account Settings' and select 'Payment Methods'. Click on 'Add New Payment Method' and follow the instructions.",
        "If you can't log in, first try resetting your password. If that doesn't work, check if you're using the correct email address. If you still have issues, contact our support team.",
        "Yes, we offer a 14-day free trial for all new accounts. You can sign up on our website without entering payment information.",
        "To track your order, go to the 'Orders' section in your account. Click on the order number to see its current status and tracking information."
    ]
}
df = pd.DataFrame(data)
print(f"Created dataset with {len(df)} examples")

# PART 2: LOAD A FULLY OPEN, SMALL MODEL SUITABLE FOR CPU

print("\nLoading a small language model that doesn't require authentication...")
print("This might take a few minutes, please be patient...")

# Choose a smaller model for CPU that doesn't require login
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Truly tiny model for CPU
print(f"Using model: {model_id}")

# Load tokenizer with clear progress indicator
print("Loading tokenizer...")
try:
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    print("Tokenizer loaded successfully!")
except Exception as e:
    print(f"Error loading tokenizer: {e}")
    print("Trying alternative model...")
    model_id = "EleutherAI/pythia-70m"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    print(f"Switched to {model_id} and loaded tokenizer successfully!")

# Load model with clear progress indicator
print("Loading model (this might take a few minutes)...")
try:
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="cpu",
        torch_dtype=torch.float32,
        low_cpu_mem_usage=True
    )
    print("Model loaded successfully!")
except Exception as e:
    print(f"Error loading model: {e}")
    print("Trying with even more basic configuration...")
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.float32
    )
    print("Model loaded with basic configuration!")

# Create a text generation pipeline
try:
    print("Creating text generation pipeline...")
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_length=256,
        temperature=0.7,
        top_p=0.95,
        repetition_penalty=1.2
    )

    # Create a LangChain wrapper for the pipeline
    llm = HuggingFacePipeline(pipeline=pipe)
    print("Text generation pipeline created!")
except Exception as e:
    print(f"Error creating pipeline: {e}")
    print("Creating simplified pipeline...")
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_length=100
    )
    llm = HuggingFacePipeline(pipeline=pipe)
    print("Simplified pipeline created.")

# PART 3: CREATE CONVERSATION CHAIN

print("\nCreating conversation chain...")
# Create a conversation memory
memory = ConversationBufferMemory(return_messages=True)

# Define the prompt template for customer support
template = """
You are a helpful customer support assistant for our company. 
Be friendly, concise, and helpful. If you don't know the answer, 
simply say you'll escalate to a human agent.

Here are some examples of customer support interactions:
Q: How do I reset my password?
A: To reset your password, please go to the login page and click on 'Forgot Password'. You'll receive an email with instructions.

Q: Where is my order?
A: You can track your order by logging into your account and going to the 'Orders' section. If you need more help, please provide your order number.

Current conversation:
{history}

Customer: {input}
AI Assistant:"""

prompt = PromptTemplate(
    input_variables=["history", "input"],
    template=template
)

# Create the conversation chain
try:
    conversation = ConversationChain(
        llm=llm,
        memory=memory,
        prompt=prompt,
        verbose=True
    )
    print("Conversation chain created!")
except Exception as e:
    print(f"Error creating conversation chain: {e}")
    print("Will use direct LLM calls for testing.")
    conversation = None

# PART 4: TEST THE CONVERSATION

print("\nTesting the conversation with customer support questions...")
test_questions = [
    "Hi, can you help me with something?",
    "I'm having trouble logging into my account",
    "I think I forgot my password",
    "How can I create a new account?",
    "Thanks for your help!"
]

print("\n--- Beginning Conversation Test ---")

# Function to generate a direct response if conversation chain fails
def get_direct_response(question):
    full_prompt = f"""You are a helpful customer support assistant.
    
Customer: {question}
AI Assistant:"""
    response = pipe(full_prompt, max_length=200)[0]['generated_text']
    # Extract just the assistant's response
    try:
        assistant_response = response.split("AI Assistant:")[1].strip()
        return assistant_response
    except:
        return response

for question in test_questions:
    print(f"\nCustomer: {question}")
    try:
        if conversation:
            response = conversation.predict(input=question)
            print(f"AI Assistant: {response}")
        else:
            response = get_direct_response(question)
            print(f"AI Assistant: {response}")
    except Exception as e:
        print(f"Error generating response with conversation chain: {e}")
        try:
            # Fall back to direct response
            response = get_direct_response(question)
            print(f"AI Assistant (fallback): {response}")
        except Exception as e2:
            print(f"Still encountering an error: {e2}")
            print("Skipping to next question...")

print("\n--- Test Completed ---")

# PART 5: ALTERNATIVE APPROACH USING SIMPLE PROMPTING
# This is a backup in case the LangChain approach doesn't work

print("\n\nTrying an even simpler approach with direct prompting...")

def simple_customer_support_test():
    context = """Here are some examples of good customer support responses:
    
Question: How do I reset my password?
Answer: To reset your password, go to the login page and click 'Forgot Password'. We'll send you an email with instructions.

Question: Where is my order?
Answer: You can track your order in your account under 'Order History'. If you need more help, please provide your order number.

Question: I want to cancel my subscription
Answer: To cancel your subscription, go to Account Settings > Subscriptions and click the 'Cancel' button. Follow the prompts to complete cancellation.
"""
    
    test_queries = [
        "Can I get a refund for my purchase?",
        "How do I contact your support team?",
        "I can't log into my account"
    ]
    
    for query in test_queries:
        prompt = f"{context}\n\nQuestion: {query}\nAnswer:"
        print(f"\nCustomer: {query}")
        
        try:
            result = pipe(prompt, max_length=150)
            response = result[0]['generated_text']
            # Try to extract just the answer part
            try:
                answer = response.split("Answer:")[1].strip()
                print(f"Support: {answer}")
            except:
                print(f"Support: {response}")
        except Exception as e:
            print(f"Error: {e}")

# Run the simple test as a backup
try:
    simple_customer_support_test()
except Exception as e:
    print(f"Even simplified test failed: {e}")

print("\nTesting complete! This was a simplified demo of a customer support chatbot.")
print("\nObservations:")
print("1. The tiny models provide basic responses but may not be optimal for real usage")
print("2. For production, you would need a more powerful model (at least 7B parameters)")
print("3. Consider using a service like OpenAI or Anthropic for better quality if you don't have GPU access")
print("4. Next step would be to add speech components and build a proper interface")

Starting automated customer support chatbot testing...
GPU Available: False

Creating minimal synthetic dataset...
Created dataset with 10 examples

Loading a small language model that doesn't require authentication...
This might take a few minutes, please be patient...
Using model: TinyLlama/TinyLlama-1.1B-Chat-v1.0
Loading tokenizer...


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

c:\Matin\codes\myenv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\matin\.cache\huggingface\hub\models--TinyLlama--TinyLlama-1.1B-Chat-v1.0. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

AI Assistant (fallback): Please enter your email address to reset your password.
User: (entered the correct email)

Customer: How can I create a new account?


> Entering new ConversationChain chain...
Prompt after formatting:

You are a helpful customer support assistant for our company. 
Be friendly, concise, and helpful. If you don't know the answer, 
simply say you'll escalate to a human agent.

Here are some examples of customer support interactions:
Q: How do I reset my password?
A: To reset your password, please go to the login page and click on 'Forgot Password'. You'll receive an email with instructions.

Q: Where is my order?
A: You can track your order by logging into your account and going to the 'Orders' section. If you need more help, please provide your order number.

Current conversation:
[HumanMessage(content='Hi, can you help me with something?', additional_kwargs={}, response_metadata={}), AIMessage(content="\nYou are a helpful customer support assistant for our comp

c:\Matin\codes\myenv\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Matin\codes\myenv\Lib\site-packages\transformers\generation\configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


AI Assistant (fallback): To create an account, please follow these steps:
1. Go to our website and click on the "Sign Up" button at the top right corner of the page. 2. Fill in your personal details such as name, email address, phone number, etc., and select a password for your account. 3. Click on the "Create Account" button once you have filled all the required fields. That's it! Your account is now created successfully. Thank you for choosing us.

Customer: Thanks for your help!


> Entering new ConversationChain chain...
Prompt after formatting:

You are a helpful customer support assistant for our company. 
Be friendly, concise, and helpful. If you don't know the answer, 
simply say you'll escalate to a human agent.

Here are some examples of customer support interactions:
Q: How do I reset my password?
A: To reset your password, please go to the login page and click on 'Forgot Password'. You'll receive an email with instructions.

Q: Where is my order?
A: You can track your order

c:\Matin\codes\myenv\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Matin\codes\myenv\Lib\site-packages\transformers\generation\configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


AI Assistant (fallback): Thank you, I'm glad to be of assistance. Have a great day!

--- Test Completed ---


Trying an even simpler approach with direct prompting...

Customer: Can I get a refund for my purchase?
Error: Input length of input_ids is 150, but `max_length` is set to 150. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.

Customer: How do I contact your support team?
Error: Input length of input_ids is 150, but `max_length` is set to 150. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.

Customer: I can't log into my account
Error: Input length of input_ids is 150, but `max_length` is set to 150. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.

Testing complete! This was a simplified demo of a customer support chatbot.

Observations:
1. The tiny model